In [1]:
import requests as req
import http.client
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
from fuzzywuzzy import process, fuzz
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Proyecto ETL Comunio

### Se pide:

###### Realizar un proyecto ETL con un tema de libre elección en la que se deben obtener datos con al menos 2 tipos técnicas de extracción
###### diferentes y desde al menos tres tipos de fuentes de datos distintas.

### Descripción del proyecto:

Vamos a realizar un proyecto de ETL sobre el fantasy de fútbol Comunio, con el que buscamos obtener los siguientes
tipos de datos.

Tabla de Equipos de La Liga. ( Id. , Nombre Equipo) .

Tabla de Jugadores. 

(Id Jugador, Id del equipo, Posición, Partidos Jugados 21/22, Goles, Asistencias, Puntos Comunio, Precio de mercado comunio, Puntos totales temporada 20/21 comunio, Pts ultimas 5 jornadas, Pts de media del campeonato,
ptos de media de las últimas 5 jornadas).

Clasificación de La Liga.

Fuentes: 

    - Tabla Equipos desde API. api.football-data.org

    - Estadisticas de jugadores (partidos jugados, goles y asistencias) web scraping desde espn.

    - Datos de comunio (Nombre, posición, valor, puntos, ... ). web scraping página comuniazo y comuniate.

- Se entrega notebook en el que se obtendran diferentes archivos con los que trabajar:
    - 1 csv con los equipos de la liga.
    - 1 csv por equipo con los datos de los jugadores de la plantilla
    - 1 csv con los datos de todos los jugadores de la liga
    - 1 json con los datos de todos los jugadores de la liga
    
------------------------------------------------------------------------------------------------------------------
### It is requested:

###### Carry out an ETL project with a subject of free choice in which data must be obtained with at least 2 types of extraction techniques
###### different and from at least three different types of data sources.

### Project description:

We are going to carry out an ETL project on the football fantasy Comunio, with which we seek to obtain the following
type of data.

La Liga Team Table. (Id., Team Name).

Players table.

(Player Id, Team Id, Position, Matches Played 21/22, Goals, Assists, Community Points, Community Market Price, Total Season 20/21 Community Points, Pts in the last 5 days, Average Pts of the championship,
average pts of the last 5 days).

Classification of La Liga

Sources:

    - Table Equipment from API. api.football-data.org

    - Player statistics (games played, goals and assists) web scraping from espn.

    - Community data (Name, position, value, points, ...). web scraping page comuniazo and comuniate.

- Notebook is delivered in which different files will be obtained with which to work:
    - 1 csv with league teams.
    - 1 csv per team with the data of the players in the squad
    - 1 csv with the data of all the players in the league
    - 1 json with the data of all the players in the league

In [2]:
# Firts we realice the connection to the API, with the instructions describes in its documentation.
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': '2643e9e1f8624a75a8eb21f12ad5c0bf' } # personal token given by the API.
connection.request('GET', '/v2/competitions/PD/teams?season=2022', None, headers ) # request for teams names
response = json.loads(connection.getresponse().read().decode())['teams'] #
team_lst = [response[t]['name'] for t in range(len(response))] # create a list of the teams
team_id = [i for i in range(1,21)] # create a list to give an id to each team
df_teams = pd.DataFrame(team_lst, columns = ['Team']) # create a Data Frame with the teams.
df_teams.insert(0, 'Team_id', team_id) # add column id to the Data Frame
df_teams.to_csv('data/temp_22-23/teams22-23.csv', index=False) # export df to a csv file.

In [3]:
#equipos con nueva fuente de datos fbref
'''
    In this cell we create a var with type list for each team with the nexts paramas:
        - index[0] : Name of the team from the var team_lst
        - index[1] : Team id from the var team_id
        - index[2] : url of comuniate that give us almost of the data 
                    (Player name, Position, Season Points, Season Points Average, Value, On start average)
        - index[3] : url of comuniazo that give us the points of the last 5 matchs of the each player on the squad
        - index[4] : url of fbref that give us the matchs played, goals, goals, assists.
        
    Then we store all of the vars in a list to give it like a param to the main function.
    
'''
ath_b_fbref = [team_lst[0], team_id[0], 'https://www.comuniate.com/plantilla/1/athletic', 'https://www.comuniazo.com/comunio-apuestas/equipos/athletic', 'https://fbref.com/es/equipos/2b390eca/Estadisticas-de-Athletic-Club']
at_m_fbref = [team_lst[1], team_id[1], 'https://www.comuniate.com/plantilla/2/atletico', 'https://www.comuniazo.com/comunio-apuestas/equipos/atletico', 'https://fbref.com/es/equipos/db3b9613/Estadisticas-de-Atletico-Madrid']
osa_fbref = [team_lst[2], team_id[2], 'https://www.comuniate.com/plantilla/12/osasuna', 'https://www.comuniazo.com/comunio-apuestas/equipos/osasuna', 'https://fbref.com/es/equipos/03c57e2b/Estadisticas-de-Osasuna']
rcd_esp_fbref = [team_lst[3], team_id[3], 'https://www.comuniate.com/plantilla/7/espanyol', 'https://www.comuniazo.com/comunio-apuestas/equipos/espanyol', 'https://fbref.com/es/equipos/a8661628/Estadisticas-de-Espanyol']
barc_fbref = [team_lst[4], team_id[4], 'https://www.comuniate.com/plantilla/3/barcelona', 'https://www.comuniazo.com/comunio-apuestas/equipos/barcelona', 'https://fbref.com/es/equipos/206d90db/Estadisticas-de-Barcelona']
geta_fbref = [team_lst[5], team_id[5], 'https://www.comuniate.com/plantilla/8/getafe', 'https://www.comuniazo.com/comunio-apuestas/equipos/getafe', 'https://fbref.com/es/equipos/7848bd64/Estadisticas-de-Getafe']
r_mad_fbref = [team_lst[6], team_id[6], 'https://www.comuniate.com/plantilla/15/real-madrid', 'https://www.comuniazo.com/comunio-apuestas/equipos/real-madrid', 'https://fbref.com/es/equipos/53a2f082/Estadisticas-de-Real-Madrid']
rayo_fbref = [team_lst[7], team_id[7], 'https://www.comuniate.com/plantilla/70/rayo-vallecano', 'https://www.comuniazo.com/comunio-apuestas/equipos/rayo-vallecano', 'https://fbref.com/es/equipos/98e8af82/Estadisticas-de-Rayo-Vallecano']
mall_fbref = [team_lst[8], team_id[8], 'https://www.comuniate.com/plantilla/11/mallorca', 'https://www.comuniazo.com/comunio-apuestas/equipos/mallorca', 'https://fbref.com/es/equipos/2aa12281/Estadisticas-de-Mallorca']
bet_fbref = [team_lst[9], team_id[9], 'https://www.comuniate.com/plantilla/4/betis', 'https://www.comuniazo.com/comunio-apuestas/equipos/betis', 'https://fbref.com/es/equipos/fc536746/Estadisticas-de-Real-Betis']
r_soc_fbref = [team_lst[10], team_id[10], 'https://www.comuniate.com/plantilla/13/real-sociedad', 'https://www.comuniazo.com/comunio-apuestas/equipos/real-sociedad', 'https://fbref.com/es/equipos/e31d1cd9/Estadisticas-de-Real-Sociedad']
villa_fbref = [team_lst[11], team_id[11], 'https://www.comuniate.com/plantilla/19/villarreal', 'https://www.comuniazo.com/comunio-apuestas/equipos/villarreal', 'https://fbref.com/es/equipos/2a8183b3/Estadisticas-de-Villarreal']
val_fbref = [team_lst[12], team_id[12], 'https://www.comuniate.com/plantilla/18/valencia', 'https://www.comuniazo.com/comunio-apuestas/equipos/valencia', 'https://fbref.com/es/equipos/dcc91a7b/Estadisticas-de-Valencia']
vall_fbref = [team_lst[13], team_id[13], 'https://www.comuniate.com/plantilla/21/real-valladolid', 'https://www.comuniazo.com/comunio-apuestas/equipos/valladolid', 'https://fbref.com/es/equipos/17859612/Estadisticas-de-Valladolid']
cadi_fbref = [team_lst[14], team_id[14], 'https://www.comuniate.com/plantilla/105/cadiz', 'https://www.comuniazo.com/comunio-apuestas/equipos/cadiz', 'https://fbref.com/es/equipos/ee7c297c/Estadisticas-de-Cadiz']
alm_fbref = [team_lst[15], team_id[15], 'https://www.comuniate.com/plantilla/22/almeria', 'https://www.comuniazo.com/comunio-apuestas/equipos/almeria', 'https://fbref.com/es/equipos/78ecf4bb/Estadisticas-de-Almeria']
elch_fbref = [team_lst[16], team_id[16], 'https://www.comuniate.com/plantilla/75/elche', 'https://www.comuniazo.com/comunio-apuestas/equipos/elche', 'https://fbref.com/es/equipos/6c8b07df/Estadisticas-de-Elche']
giro_fbref = [team_lst[17], team_id[17], 'https://www.comuniate.com/plantilla/97/girona', 'https://www.comuniazo.com/comunio-apuestas/equipos/girona', 'https://fbref.com/es/equipos/9024a00a/Estadisticas-de-Girona']
celta_fbref = [team_lst[18], team_id[18], 'https://www.comuniate.com/plantilla/5/celta', 'https://www.comuniazo.com/comunio-apuestas/equipos/celta', 'https://fbref.com/es/equipos/f25da7fb/Estadisticas-de-Celta-Vigo']
sevil_fbref = [team_lst[19], team_id[19], 'https://www.comuniate.com/plantilla/17/sevilla', 'https://www.comuniazo.com/comunio-apuestas/equipos/sevilla', 'https://fbref.com/es/equipos/ad2be733/Estadisticas-de-Sevilla']

teams_set_fbref = [ath_b_fbref, at_m_fbref, osa_fbref, rcd_esp_fbref, barc_fbref, 
                   geta_fbref, r_mad_fbref, rayo_fbref, mall_fbref,bet_fbref, 
                   r_soc_fbref, villa_fbref, val_fbref, vall_fbref, cadi_fbref, 
                   alm_fbref, elch_fbref, giro_fbref, celta_fbref, sevil_fbref]

In [17]:
def comunio_stats_fbref(team_lst, journey):
    '''
    Description.
    ------------
    
    With this function we want to obtain a dataset from diferents sources with the stadistics of all players of
    La Liga, to value its performance across the season.
    
    Params.
    -------
    
    The function receives a list with the teams of La Liga, and a the  journey's match.
    The team from teams_lst is a list with some variables:
        
        - index[0] : Name of the team from the var team_lst
        - index[1] : Team id from the var team_id
        - index[2] : url of comuniate that give us almost of the data 
                    (Player name, Position, Season Points, Season Points Average, Value, On start average)
        - index[3] : url of comuniazo that give us the points of the last 5 matchs of the each player on the squad
        - index[4] : url of FBREF that give us the matchs played, goals, assists.
    
    Journey's Macht only works from actually journey to future journeys (we can't obtains data from past journeys with this script)
    
    Returns.
    --------
        Prints the name of the team with each iteration.
        When script finish , returns 'Finished'
        Store in 'data' directory this files:
        
            1 csv file with teams of La Liga, 'data/teams.csv'.
            1 csv file with the player's squad for each team, 'data/team_name_J_number_of_journey.csv'.
            1 csv file with all the player's stats of La Liga.
            1 json file with all the player's stats of La Liga.
            1 csv file with only stats from comunio
            1 csv file with only stats from fbref
            1 json file with only stats from comunio
            1 json file with only stats from fbref
    
    '''
    
    df_teams = pd.DataFrame()
    df_comunio = pd.DataFrame()
    df_fbref = pd.DataFrame()
    
    for team in team_lst:
        try:
            print(team[0])

            soup = bs(req.get(team[2]).text , 'html.parser') # soup from comuniate web
            soup2 = bs(req.get(team[3]).text , 'html.parser') # soup from comuniazo web
            soup3 = bs(req.get(team[4]).text , 'html.parser') # soup from fbref web

            num_players = soup.find_all('div', class_='col-md-12') # Firts count the players on the squad
            total_players = 0
            for i in range (2,6):
                # This loop sum the number of player for position on the squad
                total_players += len(num_players[i].find_all('div', 
                                                             class_='enlace2 ficha_jugador col-md-6 col-sm-6 col-xs-12'))

            players = total_players

            print(total_players)


            team_id = team[1]

            squad = team[0] # this line give us the name of the team

            # With this lines get the names of the playes
            #name = soup.find_all('strong') estas dos lineas dejaron de funcionar el 13 de enero de 2022
            #name = [name[i].text for i in range(22, (22 + players * 2), 2)]
            name = soup.find_all('span', class_='titulo_ficha_jugador')
            name = [name[i].text for i in range(players)]

            # With this lines we want to get the position of the players
            pos_gk = soup.find_all('span', class_='label-posicion label-success')
            gks = [pos_gk[i].text.strip() for i in range(len(pos_gk))]

            pos_df = soup.find_all('span', class_='label-posicion label-info')
            dfs = [pos_df[i].text.strip() for i in range(len(pos_df))]

            pos_md = soup.find_all('span', class_='label-posicion label-warning')
            mds = [pos_md[i].text.strip() for i in range(len(pos_md))]

            pos_fw = soup.find_all('span', class_='label-posicion label-danger')
            fws = [pos_fw[i].text.strip() for i in range(len(pos_fw))]

            # Create an unique array for the positions of the players for arrays of the same length
            all_pos = gks + dfs + mds + fws

            # This lines get the total points on the season of the players
            pts = soup.find_all('span', class_='texto_pequenio2 label-posicion label-primary')
            pt = [pts[i].text for i in range(len(pts))]

            # This lines get the fantasy market value of the players
            values = soup.find_all('small')
            value = [values[i].text.split('€')[0].replace('.','') for i in range(0,len(values)-5,3)]

            # This line get the average points of the players along the season
            pt_avg = [values[i].text.split()[-1] for i in range(1,len(values)-5,3)]

            # This line get the percentage of the games that player are on the initial 11
            on_start_per = [values[i].text.split('%')[0] for i in range(2,len(values)-5,3)]

            # With this line get a list of the points in the last five matchs
            points = soup2.find_all('div', class_='streak')
            pts_last_five_jouyneys = [points[i].text.split() for i in range(len(points))] 
            
            if squad == 'CA Osasuna' and len(pts_last_five_jouyneys)>len(values):
                
                pts_last_five_jouyneys.drop(11)
            
            if len(pts_last_five_jouyneys[0])<5:
                for pts in pts_last_five_jouyneys:
                    
                    for i in range((5-len(pts_last_five_jouyneys[-1]))):
                        
                        pts.append(0)
            
            
            # Create variables to get a single value for each game of the last five games
            jback5 = []
            jback4 = []
            jback3 = []
            jback2 = []
            jback1 = []
            avg_last_5 = []

            # in this loop we iterate on each index of each list from pts_last_five_journeys to store the points
            # in an individual var and change type of this var to int for calculate the avg of the last 5 games
            # if the player have a '-' simbol we give o points, he don't play but we use 0 to calculate de avg

            for i, ptos in enumerate(pts_last_five_jouyneys):

                avg = 0

                try:
                    if pts_last_five_jouyneys[i][0] == '-':
                        jback5.append(0)
                    else:
                        jback5.append(int(pts_last_five_jouyneys[i][0]))
                        avg += int(pts_last_five_jouyneys[i][0])
                except:
                    jback5.append(0)

                try:
                    if pts_last_five_jouyneys[i][1] == '-':
                        jback4.append(0)
                    else:
                        jback4.append(int(pts_last_five_jouyneys[i][1]))
                        avg += int(pts_last_five_jouyneys[i][1])
                except:
                    jback4.append(0)

                try:
                    if pts_last_five_jouyneys[i][2] == '-':
                        jback3.append(0)
                    else:
                        jback3.append(int(pts_last_five_jouyneys[i][2]))
                        avg += int(pts_last_five_jouyneys[i][2])
                except:
                    jback3.append(0)

                try:
                    if pts_last_five_jouyneys[i][3] == '-':
                        jback2.append(0)
                    else:
                        jback2.append(int(pts_last_five_jouyneys[i][3]))
                        avg += int(pts_last_five_jouyneys[i][3])
                except:
                    jback2.append(0)
                try:
                    if pts_last_five_jouyneys[i][4] == '-':
                        jback1.append(0)
                    else:
                        jback1.append(int(pts_last_five_jouyneys[i][4]))
                        avg += int(pts_last_five_jouyneys[i][4])
                except:
                    jback1.append(0)

                avg_last_5.append(avg/len(pts_last_five_jouyneys[0]))        

            journey = journey

            # Create a dictionary to store stats from each player on each iteration

            team = {'Team_id': team_id, 
                    'Team': squad, 
                    'Player': name,
                    'Position': all_pos,
                    'Total_Points': pt,
                    'Points_Average': pt_avg,
                    'Value': value,
                    'On_start_%': on_start_per,
                    'Last_5_games_points': pts_last_five_jouyneys,
                    f'J_{journey}': jback5,
                    f'J_{journey - 1}': jback4,
                    f'J_{journey - 2}': jback3,
                    f'J_{journey - 3}': jback2,
                    f'J_{journey - 4}': jback1,
                    'Avg_last_5_games': avg_last_5}
            

            df = pd.DataFrame(team)
            print(f'Nº jugadores en la web de comunio {len(df.Player)}')
            df.to_csv(f'data/pruebas/{squad}_comunio-stats_J{journey}.csv', index=False)
            df_comunio = pd.concat([df_comunio,df], axis=0)

            # Second part of web scrapping to complete the stats table, with games played, goals, and assists

            players_stats = soup3.find('tbody')

            new_name = [' '.join(players_stats.find_all('tr')[i].text.split()[0:2])[:-2] if len(players_stats.find_all('tr')[i].text.split())>2
                        else ' '.join(players_stats.find_all('tr')[i].text.split()[0:1])[:-2] for i in range(len(players_stats.find_all('tr')))]

            new_pj = ['0' if players_stats.find_all('td', class_='right')[i].text == '' 
                       else players_stats.find_all('td', class_='right')[i].text 
                       for i in range (0,len(players_stats.find_all('td', class_='right')),25 )]

            new_goals = ['0' if players_stats.find_all('td', class_='right')[i].text == '' 
                       else players_stats.find_all('td', class_='right')[i].text 
                       for i in range (4,len(players_stats.find_all('td', class_='right')),25 )]

            new_assists = ['0' if players_stats.find_all('td', class_='right')[i].text == '' 
                       else players_stats.find_all('td', class_='right')[i].text 
                       for i in range (5,len(players_stats.find_all('td', class_='right')),25 )]

            pl_dict = {'Player': new_name, 'Matchs': new_pj, 'Goals': new_goals, 'Assists' : new_assists}

            df_stats = pd.DataFrame(pl_dict)

            print(f'Nº jugadores en la web de fbref {len(df_stats.Player)}')
            df_stats.to_csv(f'data/pruebas/{squad}_fbref-stats_J{journey}.csv', index=False)
            df_fbref = pd.concat([df_fbref,df_stats], axis=0)

            # In this part of script we use the fuzzywuzzy lib to merge both dataframes.

            #This part compares both dataframes and returns a score of match in the serie that we want
            # In this case we compare the serie 'Player' of both dataframes
            df[['team_matched', 'fuzz_score']]=df_stats.Player.apply(lambda x:process.extractOne(x,
                                                                         df_stats.Player.tolist(),
                                                                         scorer=fuzz.partial_ratio)).apply(pd.Series)

            # merge the both daframes
            df=pd.merge(df, df_stats, left_on='team_matched', right_on='Player')

            # Create a new datafram only with the values of fuzzy score are greater than 75

            df_1=df[df.fuzz_score>75]

            #The next line drops the columns that not are necessary on dataframe
            df_1 = df_1.drop(columns = ['team_matched', 'fuzz_score', 'Player_y'])

            # Rename the columns to the value that we want
            df_1 = df_1.rename(columns = {'Player_x': 'Player'})

            # order columns of the data frame to give sense
            order_col = [0,1,2,3,15,7,16,17,4,5,8,9,10,11,12,13,14,6]

            # Sort the dataframe columns with the value that we want
            df_1 = df_1[df_1.columns[order_col]]

            df_teams = pd.concat([df_teams,df_1], axis=0) # add the df of the team to the df of all teams

            df_1.to_csv(f'data/temp_22-23/{squad}_J{journey}.csv', index=False) # export df_team to a team file
        
        except:
            print(f'Fallo al recopilar datos de {squad}')
            print(pd.DataFrame(team))
            print('Nombre',len(name))
            print('Posiciones',len(all_pos))
            print('Puntos',len(pt))
            print('Equipo',len(name))
            print('PTs_avg',len(pt_avg))
            print('On_start',len(on_start_per))
            print('Value',len(value))
            print('Puntuacion 5 ultimas jornadas',len(pts_last_five_jouyneys))
            print('Ultima jornada',len(jback5))
            print('Penultima',len(jback4))
            print('-3',len(jback3))
            print('-4',len(jback2))
            print('-5',len(jback1))
            print('Avg_lt_5',len(avg_last_5))

    df_teams.to_json(f'data/temp_22-23/comunio_J{journey}.json', orient="table") # export df of all teams to json file
    df_teams.to_csv(f'data/temp_22-23/comunio_J{journey}.csv', index=False) # export df o all teams to csv file
    df_comunio.to_json(f'data/temp_22-23/only_comunio_stats_J{journey}.json', orient="table")
    df_fbref.to_json(f'data/temp_22-23/only_fbref_stats_J{journey}.json', orient="table")
    df_comunio.to_csv(f'data/temp_22-23/only_comunio_stats_J{journey}.csv', index=False)
    df_fbref.to_csv(f'data/temp_22-23/only_fbref_stats_J{journey}.csv', index=False)
    
    print('Jugadores en fbref ',len(df_fbref))
    print('Jugadores en comunio ',len(df_comunio))
    print('Jugadores en lista combinada ', len(df_teams))
    return 'Finished'

In [18]:
def classification(journey):
    soup5 = bs(req.get('https://fbref.com/es/comps/12/Estadisticas-de-La-Liga').text, 'html.parser')
    
    clas_liga = soup5.find('tbody')
    pos = [clas_liga.find_all('th')[i].text for i in range(len(clas_liga.find_all('th')))]
    team = [clas_liga.find_all('td', class_='left')[i].text for i in range(0,len(clas_liga.find_all('td', class_='left')),3)]
    pj = [clas_liga.find_all('td', class_='right')[i].text for i in range(0,len(clas_liga.find_all('td', class_='right')),16)]
    pg = [clas_liga.find_all('td', class_='right')[i].text for i in range(1,len(clas_liga.find_all('td', class_='right')),16)]
    pe = [clas_liga.find_all('td', class_='right')[i].text for i in range(2,len(clas_liga.find_all('td', class_='right')),16)]
    pp = [clas_liga.find_all('td', class_='right')[i].text for i in range(3,len(clas_liga.find_all('td', class_='right')),16)]
    gf = [clas_liga.find_all('td', class_='right')[i].text for i in range(4,len(clas_liga.find_all('td', class_='right')),16)]
    gc = [clas_liga.find_all('td', class_='right')[i].text for i in range(5,len(clas_liga.find_all('td', class_='right')),16)]
    dg = [clas_liga.find_all('td', class_='right')[i].text for i in range(6,len(clas_liga.find_all('td', class_='right')),16)]
    pts = [clas_liga.find_all('td', class_='right')[i].text for i in range(7,len(clas_liga.find_all('td', class_='right')),16)]

    clas_tab = {'Position': pos, 'Team': team, 'Games': pj, 'Won': pg, 'Draw': pe, 'Lost': pp, 'Goals S.': gf, 'Goals R.': gc, 'GD': dg, 'PTS': pts}
    
    clas_df = pd.DataFrame(clas_tab)
    
    clas_df.to_excel(f'data/temp_22-23/classification_J_{journey}.xlsx', sheet_name=f'classification_J_{journey}')
    
    return 'Classification done'
    
    
    

In [19]:
def calendar_season_as():    
    soup6 = bs(req.get('https://resultados.as.com/resultados/futbol/primera/calendario/').text, 'html.parser')
    matches = soup6.find_all('span', class_="nombre-equipo")
    jornadas = [i for i in range(1,39)]
    journey_lst = []
    for j in jornadas:
        for i in range(1,11):
            journey_lst.append(j)
            
    home = [soup6.find_all('span', class_='nombre-equipo')[i].text for i in range(0,len(soup6.find_all('span', class_='nombre-equipo')),2)]
    away = [soup6.find_all('span', class_='nombre-equipo')[i].text for i in range(1,len(soup6.find_all('span', class_='nombre-equipo')),2)]
    matches_list = {'Journey': journey_lst, 'Home': home, 'Away': away}

    season_calendar = pd.DataFrame(matches_list)

    season_calendar.to_excel('data/temp_22-23/season_calendar_22-23.xlsx', sheet_name='Season_22-23')

    return 'Season 22-23 done'

In [20]:
#calendar_season()

In [21]:
#comunio_stats_fbref(teams_set_fbref, 28)

In [22]:
comunio_stats_fbref(teams_set_fbref, 3)

Athletic Club
26
Nº jugadores en la web de comunio 26
Nº jugadores en la web de fbref 24
Club Atlético de Madrid
22
Nº jugadores en la web de comunio 22
Nº jugadores en la web de fbref 24
CA Osasuna
26


,Team_id,Team,Player,Position,Total_Points,Points_Average,Value,On_start_%,Last_5_games_points,J_3,J_2,J_1,J_0,J_-1,Avg_last_5_games
0,3,CA Osasuna,Sergio Herrera,PT,12,4,4080000,100,"[4, 6, 2, 0, 0]",4,6,2,0,0,2.4
1,3,CA Osasuna,Juan Pérez,PT,0,0,180000,0,"[-, -, -, 0, 0]",0,0,0,0,0,0.0
2,3,CA Osasuna,Aitor Fernández,PT,0,0,1040000,0,"[-, -, -, 0, 0]",0,0,0,0,0,0.0
3,3,CA Osasuna,David García Zub.,DF,15,5,4990000,100,"[3, 7, 5, 0, 0]",3,7,5,0,0,3.0
4,3,CA Osasuna,Rubén Peña,DF,14,4.7,2290000,100,"[5, 7, 2, 0, 0]",5,7,2,0,0,2.8
5,3,CA Osasuna,Unai García,DF,13,4.3,990000,100,"[3, 8, 2, 0, 0]",3,8,2,0,0,2.6
6,3,CA Osasuna,Juan Cruz,DF,10,3.3,1390000,100,"[4, 5, 1, 0, 0]",4,5,1,0,0,2.0
7,3,CA Osasuna,Nacho Vidal,DF,9,3,2530000,33,"[2, 2, 5, 0, 0]",2,2,5,0,0,1.8
8,3,CA Osasuna,Manu Sánchez,DF,7,3.5,1620000,0,"[3, -, 4, 0, 0]",3,0,4,0,0,1.4
9,3,CA Osasuna,Unai Dufur,DF,0,0,220000,0,"[-, -, -, 0, 0]",0,0,0,0,0,0.0


Nº jugadores en la web de comunio 26
Nº jugadores en la web de fbref 25
RCD Espanyol de Barcelona
27
Nº jugadores en la web de comunio 27
Nº jugadores en la web de fbref 24
FC Barcelona
29
Nº jugadores en la web de comunio 29
Nº jugadores en la web de fbref 25
Getafe CF
23
Nº jugadores en la web de comunio 23
Nº jugadores en la web de fbref 25
Real Madrid CF
24
Nº jugadores en la web de comunio 24
Nº jugadores en la web de fbref 25
Rayo Vallecano de Madrid
25
Nº jugadores en la web de comunio 25
Nº jugadores en la web de fbref 25
RCD Mallorca
26
Nº jugadores en la web de comunio 26
Nº jugadores en la web de fbref 25
Real Betis Balompié
28
Nº jugadores en la web de comunio 28
Nº jugadores en la web de fbref 24
Real Sociedad de Fútbol
27
Nº jugadores en la web de comunio 27
Nº jugadores en la web de fbref 27
Villarreal CF
25
Nº jugadores en la web de comunio 25
Nº jugadores en la web de fbref 24
Valencia CF
29
Nº jugadores en la web de comunio 29
Nº jugadores en la web de fbref 28
Real V

'Finished'

In [8]:
classification(3)

'Classification done'

In [ ]:
calendar_season_as()